In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [29]:
complaints = pd.read_csv('../data/complaints.csv')

In [30]:
# Replacing the X's with an empty space.
for i in range(0, len(complaints)):
    complaints.loc[i, 'Consumer complaint narrative'] = (
        re.sub('X{2,}', '', complaints.loc[i, 'Consumer complaint narrative'])
    )

In [31]:
# Creating the Train/Test spilt stratifying by Issue.
X = complaints[['Consumer complaint narrative']]
y = complaints['Issue']

xTrain, xTest, yTrain, yTest = train_test_split(X, y, random_state = 777, stratify = y)

**Looking at a Count Vectorizer**

In [32]:
# Fitting the count vectorizer to training data then transforming both the training and test data.
vect = CountVectorizer()

XTrainVec = vect.fit_transform(xTrain['Consumer complaint narrative'])
XTestVec = vect.transform(xTest['Consumer complaint narrative'])

In [33]:
# Checking the results of a logistic regression model.
logreg = LogisticRegression(max_iter = 1000).fit(XTrainVec, yTrain)

yPred = logreg.predict(XTestVec)

print('Accuracy Score: ', accuracy_score(yTest, yPred))

print('Confusion Matrix: ', confusion_matrix(yTest, yPred))

C:\Users\hcgre\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Score:  0.8713868580094615
Confusion Matrix:  [[12223   788   118  5071    91]
 [  933  3987    24   336    31]
 [  158    29  2630   251    19]
 [ 2683   105   102 54258   178]
 [   92    54    22   279  3896]]


In [38]:
# Data frame of word counts in the training set
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': np.array(XTrainVec.sum(axis = 0)).flatten()
})

word_counts.sort_values('frequency', ascending = False).head()

,words,frequency
61571,the,1829587
62727,to,1488528
7488,and,1228351
41851,my,961732
44061,of,888830
